# Worksheet 08

Name: Victor Verma
UID: U86967149

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [1]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)
print(np.mean(s1))

[3.015266774698381, 4.621614555950935, 4.244614782596882, 5.3643633477725405, 4.84676009357638, 3.489650359673156, 4.697411141254303, 4.915396104681839, 5.201175663768233, 4.472850926956348]
4.4869103750929


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [2]:
s2 = np.random.normal(8, 1, 10).tolist()
print(s2)
print(np.mean(s2))

[8.449862924114997, 6.943838763680527, 7.5101154037552575, 9.082465743286216, 7.188580191023628, 8.770296645414225, 8.803420343803595, 8.51304190432293, 7.260666685833643, 7.2627245069196285]
7.978501311215465


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `s1` and remove it from `s1`, if T then pick the last data point from `s2` and remove it from `s2`. Add these 10 points to a list called `data`.

In [3]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[4.472850926956348, 7.2627245069196285, 7.260666685833643, 8.51304190432293, 8.803420343803595, 8.770296645414225, 7.188580191023628, 5.201175663768233, 4.915396104681839, 9.082465743286216]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

- Prior probability of species 1
- Prior probability of species 2
- Mean of species 1
- Mean of species 2
- Variance of species 1
- Variance of species 2

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [21]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)), len(s2) / (len(s1) + len(s2)) ]
mean = [ sum(s1)/len(s1), sum(s2)/len(s2)]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1), sum(map(lambda x : (x - mean[0])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[7.2627245069196285, 7.260666685833643, 8.51304190432293, 8.803420343803595, 8.770296645414225, 7.188580191023628, 9.082465743286216]
[4.472850926956348, 5.201175663768233, 4.915396104681839]
P(S_1) = 0.7,  P(S_2) = 0.3
mean_1 = 8.12588514580055,  mean_2 = 4.8631408984688065
var_1 = 0.6159209672504508,  var_2 = 10.735274813826587


The estimates for mean_1 and mean_2 are 8.12 and 4.86, while the estimates for var_1 and var_2 are 0.61 and 10.73. The real mean values for these parameters are 7.98 and 4.49. I would say that the initial estimates are really close. However, I would imagine that there would be variation depending on the overlap of the clusters, with some configurations having better initial estimates than others.

e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [22]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append(pdf_i[0] * prob_s[0] / prob_x)
    prob_s1_x.append(pdf_i[1] * prob_s[1] / prob_x)

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  4.472850926956348
probability of observing that point if it came from cluster 0 =  1.488741739472644e-08
probability of observing that point if it came from cluster 1 =  0.03713726233963731
point =  7.2627245069196285
probability of observing that point if it came from cluster 0 =  0.24261405317020854
probability of observing that point if it came from cluster 1 =  0.03624496196648836
point =  7.260666685833643
probability of observing that point if it came from cluster 0 =  0.24147939496073864
probability of observing that point if it came from cluster 1 =  0.03624651431008765
point =  8.51304190432293
probability of observing that point if it came from cluster 0 =  0.5316023675539241
probability of observing that point if it came from cluster 1 =  0.035074860273943985
point =  8.803420343803595
probability of observing that point if it came from cluster 0 =  0.3536875296057521
probability of observing that point if it came from cluster 1 =  0.034741064120873945
point =  8.77

For the most part, it seems that for each point, the cluster with a higher probability of being correct, is indeed the true correct cluster. However, the large variance of cluster 1, compared to cluster 0, is causing issues because it is allowing a greater range of points to be classified as cluster 1. I think that the accuracy of the first round of clustering would greatly depend on the initial values of the means and variances as determined by k-means.

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [23]:
prob_c = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/ len(prob_s1_x)]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)]
var = [sum([x[0] * (x[1] - mean[0]) ** 2 for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * (x[1] - mean[1]) ** 2 for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.7,  P(S_2) = 0.3
mean_1 = 8.131746430523842,  mean_2 = 5.207997677479457
var_1 = 0.6122465077325174,  var_2 = 1.1230095242592022


Both of the means actually got a little farther from the true value, having both increased. However, variance 2 went down a great deal, and I think that this would have a much greater impact in the next round of clustering. I would expect an algorithm like this to require many iterations to reach an optimal clustering, so I am not too worried about the means getting worse.

g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [24]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append(pdf_i[0] * prob_s[0] / prob_x)
    prob_s1_x.append(pdf_i[1] * prob_s[1] / prob_x)

probs = zip(data, prob_s0_x, prob_s1_x)

prob_c = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/ len(prob_s1_x)]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)]
var = [sum([x[0] * (x[1] - mean[0]) ** 2 for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * (x[1] - mean[1]) ** 2 for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))



point =  4.472850926956348
probability of observing that point if it came from cluster 0 =  1.14453352495967e-08
probability of observing that point if it came from cluster 1 =  0.28672959727226094
point =  7.2627245069196285
probability of observing that point if it came from cluster 0 =  0.237957155089597
probability of observing that point if it came from cluster 1 =  0.06661799512613757
point =  7.260666685833643
probability of observing that point if it came from cluster 0 =  0.23682328941282563
probability of observing that point if it came from cluster 1 =  0.06684160795913126
point =  8.51304190432293
probability of observing that point if it came from cluster 0 =  0.5367374015762872
probability of observing that point if it came from cluster 1 =  0.004674437409829773
point =  8.803420343803595
probability of observing that point if it came from cluster 0 =  0.3569735775134501
probability of observing that point if it came from cluster 1 =  0.002112129116808064
point =  8.77029

Both of the means got a little closer to their true values. The second variance got smaller again. I am not sure how many iterations are usually necessary to get the optimal results. We are only at 2 iterations, so I would think that more are needed.

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [27]:
assignments = {}
for p in range(len(data)):
    if prob_s0_x[p] > prob_s1_x[p]:
        assignments[data[p]] = 0
    else:
        assignments[data[p]] = 1
print(assignments)

{4.472850926956348: 1, 7.2627245069196285: 0, 7.260666685833643: 0, 8.51304190432293: 0, 8.803420343803595: 0, 8.770296645414225: 0, 7.188580191023628: 0, 5.201175663768233: 1, 4.915396104681839: 1, 9.082465743286216: 0}
